In [33]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.models import load_model

In [3]:
# Load model
model = load_model('../model/pitching_model.h5')
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ batter_input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 10)     │     12,460 │ batter_input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 10)        │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ other_input         │ (None, 108)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 118)       │          0 │ flatten_3[0][0],  │
│ (Concatenate)       │                   │            │ other_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │      7,616 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 32)        │      2,080 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         33 │ dense_10[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 22,191 (86.69 KB)

 Trainable params: 22,189 (86.68 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Scenario: Rockies are playing the Yankees in the home opener, and the Rockies want to show how AI and Moneyball can come together to make the ultimate movie: pitch perfect.

Gleyber Torres is up to bat. First pitch of the day, German Marquez on the mound, what should we throw?

In [11]:
# Bring in the csv 
path = '../data/model_testing_data.csv'
test_data = pd.read_csv(path)
test_data = test_data.head(8)
test_data 

,Unnamed: 0,batter,zone,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,...,of_fielding_alignment_Strategic,trailing_pitch_CU,trailing_pitch_FC,trailing_pitch_FF,trailing_pitch_KC,trailing_pitch_PO,trailing_pitch_SI,trailing_pitch_SL,trailing_pitch_ST,trailing_pitch_none
0,0.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,5.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,6.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,7.0,429.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Data is consistent accross the rows except for the pitch thrown: 

changeup
curve
four seam
knucle
pitchout
two seam
slider
stinker


In [13]:
# Getting the data ready.

def preprocess_data_function(path):
    df = pd.read_csv(path)
    df = test_data.head(8)

    # prepare batter feature
    X_batter_new = df[['batter']].values
    X_other_new = df.drop(columns= ['batter']).values


    return X_batter_new, X_other_new

In [43]:
# Predicting
def run_predictions_function(X_batter_new,X_other_new):
    
    y_pred_new = model.predict([X_batter_new,X_other_new])
    y_pred_classes_new = (y_pred_new > 0.5).astype(int)
    
    # Creating DF
    X_batter_df = pd.DataFrame(X_batter_new, columns=['batter_id'])  # Adjust column names as needed
    X_other_df = pd.DataFrame(X_other_new, columns=[f'feature_{i}' for i in range(X_other_new.shape[1])])  # Use appropriate names for features

    # Concatenate X_batter and X_other into one DataFrame
    X_combined_df = pd.concat([X_batter_df.reset_index(drop=True), X_other_df.reset_index(drop=True)], axis=1)

    # Create a DataFrame with Ys and predictions
    results_df = pd.DataFrame({
    'True Labels': y_pred_new.flatten(),  # Flatten if necessary
    'Predictions': y_pred_classes_new.flatten()  # Flatten predictions
    })

    #  Combine the results with the input data
    final_results_df = pd.concat([X_combined_df, results_df], axis=1)
    final_results_df = final_results_df[['Predictions'] + [col for col in final_results_df.columns if col != 'Predictions']]
    return final_results_df

In [44]:
def model_load_and_run(path):
    X_batter_new, X_other_new = preprocess_data_function(path)
    df = run_predictions_function(X_batter_new, X_other_new)
    
    return df

In [45]:
model_load_and_run(path)

array([[429.],
       [429.],
       [429.],
       [429.],
       [429.],
       [429.],
       [429.],
       [429.]])

array([[0.  , 8.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 3.27, 1.67,
        1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ,
        1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 1.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ],
       [1.  , 8.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 3.27, 1.67,
        1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ,
        1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  ,

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


,Predictions,batter_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,True Labels
0,0,429.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.849870e-13
1,0,429.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.272606e-13
2,0,429.0,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.857821e-13
3,0,429.0,3.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.935884e-14
4,0,429.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.870262e-13
5,0,429.0,5.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.102437e-13
6,0,429.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.257746e-13
7,0,429.0,7.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.977360e-13


Well, looks like we cant throw a strike off rip huh. classic rockies.